In [9]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
requests.packages.urllib3.disable_warnings()
import urllib.request
import ssl
import re
months = {'Января':'01', 'Февраля':'02','Марта':'03','Апреля':'04','Мая':'05','Июня':'06','Июля':'07',
          'Августа':'08','Сентября':'09','Октября':'10','Ноября':'11','Декабря':'11'}


In [10]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

baseurl = 'https://reestr.digital.gov.ru/'

In [11]:
#поиск количества страниц c отображением по 100
def find_n():
    req = requests.get('https://reestr.digital.gov.ru/reestr/?show_count=100', verify=False).text
    res = BeautifulSoup(req, 'lxml').find('div', class_='page_nav_area').find_all('a', class_='nav_item')
    return int(res[3].text)+1

In [12]:
for i in range(1, find_n()):
    # Создание ссылок для всех страниц поиска на сайте реестра (с отображением по 100)
    url = baseurl + '/reestr/?PAGEN_1=' + str(i) + '&show_count=100'
    response = urllib.request.urlopen(url)
    print(url)
    # поиск ссылок на страницы с юридической информации о ПО
    links = re.findall('<a href=\"(/reestr/\d*/)\"', str(response.read()))
    with open('links.txt', mode='a', encoding='utf-8') as myfile:
        for line in links:
            print(baseurl + line, file=myfile)
    myfile.close

https://reestr.digital.gov.ru//reestr/?PAGEN_1=1&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=2&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=3&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=4&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=5&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=6&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=7&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=8&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=9&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=10&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=11&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=12&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=13&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=14&show_count=100
https://reestr.digital.gov.ru//reestr/?PAGEN_1=15&show_count=100
https://reestr.digital.gov.ru//ree

In [17]:
# константные значения
links = "links.txt"
# количество уникальных названий ПО
num = 1000

# создание словаря для передачи его в последствии датафрейму в пандас
values = {'Название по': [], 'Ссылка на ПО':[], 'Вид организации':[],'Название организации': [], 'Все продукты организации':[], 'ИНН': [],'Сведения об исключительном праве':[], 'Сайт': [], 'Альтернатив': [],  'Класс по': [], 'Дата регистрации':[], 'Рег.номер ПО':[],
         'Дата решения':[],'Решение':[],'Ссылка на приказ':[]}

values['Название по'] = ['' for element in range(num)]
values['Ссылка на ПО'] = ['' for element in range(num)]
values['Название организации'] = ['' for element in range(num)]
values['ИНН'] = ['' for element in range(num)]
values['Сведения об исключительном праве'] = ['' for element in range(num)]
values['Сайт'] = ['' for element in range(num)]
values['Альтернатив'] = ['' for element in range(num)]
values['Класс по'] = ['' for element in range(num)]
values['Вид организации'] = ['' for element in range(num)]
values['Все продукты организации'] = ['' for element in range(num)]
values['Дата регистрации'] = ['' for element in range(num)]
values['Рег.номер ПО'] = ['' for element in range(num)]
values['Дата решения'] = ['' for element in range(num)]
values['Решение'] = ['' for element in range(num)]
values['Ссылка на приказ'] = ['' for element in range(num)]



In [18]:
f = open(links)
i = 0
for url in f:
    print(url)
    print(str(i) + ' of ' + str(num))
    req = requests.get(url, verify=False).text
    soup = BeautifulSoup(req, 'lxml')
    # 1 название ПО
    soft = soup.find('h1', id='pagetitle').text
    values['Название по'][i] = soft
    #15 ссылка на ПО
    values['Ссылка на ПО'][i] = url
    # 2 название организации
    try:
        orgname = soup.find_all('a', title=re.compile("Все продукты"))
    except AttributeError:
        orgname = ""
        pass
    values['Название организации'][i] = '::'.join([i.text.strip() for i in orgname])
    # 3 ИНН ^(\d{10}|\d{12})$
    try:
        inn = soup.find_all('div', text=re.compile("(\d{10}\s|\d{12}\s)"))
    except AttributeError:
        inn = ''
        pass
    #инн может быть несколько, например в случае 4 правообладателей-людей, поэтому инн в списке
    values['ИНН'][i] = '::'.join([i.text.strip() for i in inn])
    # 4 альтернативное название продукта
    try:
        altname = str(soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div')).split('<br/>')
        altname = [x.strip() for x in altname]
        altname = re.sub('''<div style="margin-bottom: 16px;">\n<span style="font-size: 15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>''','',
                         altname[0]).strip() +'::'+ "::".join(altname[1:-1])
        altname = re.sub('</div>','', altname.strip('::')).strip()
    except AttributeError:
        altname = ""
        pass
    values['Альтернатив'][i] = altname.strip()
    print(altname.strip())
    # 5 Класс по
    try:
        softclas = soup.find('span', text=re.compile("Класс ПО:")).find_parent('div').find_all('font')
        softclass = '::'.join([x.text for x in softclas])
    except AttributeError:
        softclass = ""
        pass
    values['Класс по'][i] = softclass
    # 6 сайт производителя
    try:
        site = soup.find('span', text=re.compile("Сайт производителя:")).find_parent('div').find('a').get('href')
    except AttributeError:
        site = ""
        pass
    values['Сайт'][i] = site
    # 7 вид организации
    try:
        type_of_organization = soup.find("h5", attrs={ "style" : "font-size: 20px; margin-bottom: 12px;"}).text
    except AttributeError:
        type_of_organization = ""
        pass
    values['Вид организации'][i] = type_of_organization
    # 8 все продукты организации
    try:
        all_products = soup.find('a', title='Все продукты организации').get('href')
    except AttributeError:
        all_products = ""
        pass
    values['Все продукты организации'][i] = 'https://reestr.digital.gov.ru' + all_products
    # 9 дата регистрации
    try:
        date_of_registration = soup.find('span', text=re.compile("Дата регистрации:")).find_parent('div').text
        temp = re.sub('Дата регистрации:', '', date_of_registration).strip()
        temp2 = ' '.join([x for x in temp.split(' ') if x!='']).split(' ')
        temp3 = '.'.join([temp2[0],months.get(temp2[1]),temp2[2]])
    except AttributeError:
        temp3 = ""
        pass
    values['Дата регистрации'][i] = temp3
    # 10 регистрационный номер ПО
    try:
        reg_number = soup.find('span', text=re.compile("Рег. номер ПО:")).find_parent('div').text
        r = re.sub('Рег. номер ПО:', '', reg_number).strip()
    except AttributeError:
        r = ""
        pass
    print(reg_number)
    values['Рег.номер ПО'][i] = r
    # 11 дата решения
    try:
        date_of_decision = soup.find('span', text=re.compile("Дата решения")).find_parent('div').text
        tempp = re.sub('Дата решения уполномоченного органа:', '', date_of_decision).strip()
        tempp2 = ' '.join([x for x in tempp.split(' ') if x!='']).split(' ')
        tempp3 = '.'.join([tempp2[0],months.get(tempp2[1]),tempp2[2]])
    except AttributeError:
        tempp3 = ""
        pass
    values['Дата решения'][i] = tempp3
    # 12 решение
    try:
        decision = soup.find('span', text=re.compile("Решение уполномоченного")).find_parent('div').text
    except AttributeError:
        decision = ""
        pass
    values['Решение'][i] = re.sub('Решение уполномоченного органа:', '', decision).strip()
    # 13 ссылка на приказ минкомсвязи
    try:
        link = soup.find('span', text=re.compile("Ссылка на приказ")).find_parent('div').find('a').get('href')
    except AttributeError:
        link = ""
        pass
    values['Ссылка на приказ'][i] = link
    # 14 сведения об исключительном праве
    try:
        svedeniya = soup.find('p', attrs = {'style':"line-height: 28px;"}).text.strip().replace('\n', '')
    except AttributeError:
        svedeniya = ''
        pass
    values['Сведения об исключительном праве'][i] = svedeniya
        
    i = i + 1
    if i>num-1:
        break


https://reestr.minsvyaz.ru/reestr/58191/

0 of 1000
СУБД «Ред База Данных»

Рег. номер ПО:
                1            
https://reestr.minsvyaz.ru/reestr/58193/

1 of 1000


Рег. номер ПО:
                2            
https://reestr.minsvyaz.ru/reestr/58192/

2 of 1000


Рег. номер ПО:
                3            
https://reestr.minsvyaz.ru/reestr/61242/

3 of 1000


Рег. номер ПО:
                5            
https://reestr.minsvyaz.ru/reestr/61245/

4 of 1000
ЭПС "Система ГАРАНТ"

Рег. номер ПО:
                6            
https://reestr.minsvyaz.ru/reestr/61250/

5 of 1000
Е1 Евфрат

Рег. номер ПО:
                8            
https://reestr.minsvyaz.ru/reestr/61248/

6 of 1000
Альт Линукс СПТ, сертификат ФСТЭК

Рег. номер ПО:
                9            
https://reestr.minsvyaz.ru/reestr/61251/

7 of 1000


Рег. номер ПО:
                10            
https://reestr.minsvyaz.ru/reestr/61252/

8 of 1000


Рег. номер ПО:
                11            
https://reestr.minsvyaz

Dr.Web DSS::DSS::Dr.Web Desktop Security Suite (macOS) версия 12::Dr.Web Desktop Security Suite (для Windows) версия 11::Dr.Web Desktop Security Suite (для Linux) версия 11::Dr.Web Desktop Security Suite (Windows) версия 12

Рег. номер ПО:
                47            
https://reestr.minsvyaz.ru/reestr/61321/

44 of 1000
Dr.Web ESS::ESS::Dr.Web Enterprise Security Suite версия 10::Dr.Web Enterprise Security Suite версия 6.0::Dr.Web Enterprise Suite. Антивирусная защита для рабочих станций и серверов Windows версия 5.0::Dr.Web Enterprise Security Suite версия 12

Рег. номер ПО:
                48            
https://reestr.minsvyaz.ru/reestr/61322/

45 of 1000


Рег. номер ПО:
                49            
https://reestr.minsvyaz.ru/reestr/61323/

46 of 1000


Рег. номер ПО:
                50            
https://reestr.minsvyaz.ru/reestr/61324/

47 of 1000


Рег. номер ПО:
                51            
https://reestr.minsvyaz.ru/reestr/61325/

48 of 1000


Рег. номер ПО:
           

ДСПИ «CROS»

Рег. номер ПО:
                109            
https://reestr.minsvyaz.ru/reestr/65279/

106 of 1000
FLEXTERA Framework

Рег. номер ПО:
                110            
https://reestr.minsvyaz.ru/reestr/65122/

107 of 1000
Программа для ЭВМ "Средство защиты информации "vGate R2""

Рег. номер ПО:
                113            
https://reestr.minsvyaz.ru/reestr/65123/

108 of 1000
Программа для ЭВМ "Средство защиты информации "vGate-S R2""

Рег. номер ПО:
                114            
https://reestr.minsvyaz.ru/reestr/65124/

109 of 1000


Рег. номер ПО:
                115            
https://reestr.minsvyaz.ru/reestr/65125/

110 of 1000


Рег. номер ПО:
                116            
https://reestr.minsvyaz.ru/reestr/65126/

111 of 1000


Рег. номер ПО:
                117            
https://reestr.minsvyaz.ru/reestr/65127/

112 of 1000


Рег. номер ПО:
                118            
https://reestr.minsvyaz.ru/reestr/65128/

113 of 1000
ИСС "Техэксперт"::Профессиональ

Call-центр::Колл-центр::программное обеспечение для построения контактных центров::программное обеспечение центра телефонного обслуживания::программное обеспечение центра обработки вызовов::программное обеспечение обработки входящих вызовов (звонков)::программное обеспечение автоматизации исходящего обзвона

Рег. номер ПО:
                177            
https://reestr.minsvyaz.ru/reestr/65254/

171 of 1000
InfoWatch CryptoStorage SОНО 2.0::InfoWatch CryptoStorage SDK 1.0

Рег. номер ПО:
                178            
https://reestr.minsvyaz.ru/reestr/65256/

172 of 1000
InfoWatch  Traffic Monitor Standard Edition::Traffic Monitor 3.5 Standart

Рег. номер ПО:
                179            
https://reestr.minsvyaz.ru/reestr/65240/

173 of 1000
Traffic Monitor 3.5

Рег. номер ПО:
                180            
https://reestr.minsvyaz.ru/reestr/65243/

174 of 1000
Kaspersky Fraud Prevention Mobile SDK

Рег. номер ПО:
                181            
https://reestr.minsvyaz.ru/reestr/652

СПС КонсультантПлюс::ДИУ СПС КонсультантПлюс: Москва Проф::ДИУ СПС Консультант Бюджетные организации::ДИУ СПС Консультант Бизнес::ДИУ СПС Консультант Бухгалтер смарт-комплект Оптимальный::ДИУ СПС Консультант Универсал смарт-комплект Оптимальный::ДИУ СПС Кодексы РФ::ДИУ СС КонсультантБухгалтер: Версия Проф::ДИУ СС КонсультантБухгалтер: Вопросы-ответы::ДИУ СС Перспективы и риски споров в суде общей юрисдикции::ДИУ СС КонсультантАрбитраж: Налоговые споры::ДИУ СС Проверки и штрафы::ДИУ СС НТА по охране труда и пожарной безопасности в офисах и ТЦ::ДИУ ОП КонсультантПлюс: Конструктор договоров Модуль доступа::СПС/БД КонсультантПлюс: Практика антимонопольной службы::СПС/БД КонсультантПлюс: Эксперт-приложение::СПС/БД КонсультантПлюс: Сводное региональное законодательство::СПС/БД КонсультантПлюс: Московский выпуск::СПС/БД КонсультантПлюс: Московская область::СПС/БД КонсультантПлюс: Международное право::СПС/БД КонсультантПлюс: Санкт-Петербург и Ленинградская область::СПС/БД КонсультантМедицинаФа

...

Незабудка

Рег. номер ПО:
                719            
https://reestr.minsvyaz.ru/reestr/73734/

687 of 1000


Рег. номер ПО:
                720            
https://reestr.minsvyaz.ru/reestr/73735/

688 of 1000
iSpring Free

Рег. номер ПО:
                721            
https://reestr.minsvyaz.ru/reestr/73736/

689 of 1000
Программа построения систем поддержки принятия решений Галактика BI

Рег. номер ПО:
                722            
https://reestr.minsvyaz.ru/reestr/73737/

690 of 1000


Рег. номер ПО:
                723            
https://reestr.minsvyaz.ru/reestr/73738/

691 of 1000
iSpring Pro

Рег. номер ПО:
                724            
https://reestr.minsvyaz.ru/reestr/73739/

692 of 1000
iSpring QuizMaker

Рег. номер ПО:
                725            
https://reestr.minsvyaz.ru/reestr/73740/

693 of 1000
iSpring Suite

Рег. номер ПО:
                726            
https://reestr.minsvyaz.ru/reestr/73741/

694 of 1000
Галактика РУЗ

Рег. номер ПО:
                7

Staply

Рег. номер ПО:
                796            
https://reestr.minsvyaz.ru/reestr/75117/

757 of 1000
АРМ декларанта «Феанор»

Рег. номер ПО:
                797            
https://reestr.minsvyaz.ru/reestr/75118/

758 of 1000
ППД «Контроль ДТ»::Программа проверки достоверности  «Контроль ГТД»::ППД «Контроль ГТД»

Рег. номер ПО:
                798            
https://reestr.minsvyaz.ru/reestr/75119/

759 of 1000
KPI MONITOR Analyzer::KPI MONITOR Basic::KPI MONITOR Mobile

Рег. номер ПО:
                799            
https://reestr.minsvyaz.ru/reestr/75121/

760 of 1000
Программный комплекс "Гектор: Проектировщик-строитель"::Программа "Гектор: Проектировщик-строитель"

Рег. номер ПО:
                801            
https://reestr.minsvyaz.ru/reestr/75122/

761 of 1000


Рег. номер ПО:
                802            
https://reestr.minsvyaz.ru/reestr/75123/

762 of 1000


Рег. номер ПО:
                803            
https://reestr.minsvyaz.ru/reestr/75124/

763 of 1000


Рег

Составление и исполнение доходов и расходов бюджетов субъектов, ЗАТО и муниципальных образований в технологии СМАРТ с расширенным функци...::Бюджет-КС

Рег. номер ПО:
                864            
https://reestr.minsvyaz.ru/reestr/75814/

821 of 1000
Автоматизация местного самоуправления для администрации муниципальных районов, городских округов, городских и сельских поселений («Муни...::Муниципальное самоуправление - КС::МСУ-КС

Рег. номер ПО:
                865            
https://reestr.minsvyaz.ru/reestr/75815/

822 of 1000
Система управления ресурсами

Рег. номер ПО:
                866            
https://reestr.minsvyaz.ru/reestr/75816/

823 of 1000
IndorCAD/River: Система подготовки лоцманских карт и проектирования русловых работ

Рег. номер ПО:
                867            
https://reestr.minsvyaz.ru/reestr/75817/

824 of 1000
ЛИС "АЛИСА"::ALISA

Рег. номер ПО:
                868            
https://reestr.minsvyaz.ru/reestr/75818/

825 of 1000
IndorDraw: Система подгото

КСЗИ "Панцирь+" для ОС Microsoft Windows::КСЗИ "Панцирь+"

Рег. номер ПО:
                926            
https://reestr.minsvyaz.ru/reestr/75876/

882 of 1000
СЗ "Панцирь+"::Система защиты от запуска вредоносного ПО и от модификации санкционированных исполняемых объектов "Панцирь+"

Рег. номер ПО:
                927            
https://reestr.minsvyaz.ru/reestr/75877/

883 of 1000


Рег. номер ПО:
                928            
https://reestr.minsvyaz.ru/reestr/75878/

884 of 1000


Рег. номер ПО:
                929            
https://reestr.minsvyaz.ru/reestr/75879/

885 of 1000


Рег. номер ПО:
                930            
https://reestr.minsvyaz.ru/reestr/75880/

886 of 1000


Рег. номер ПО:
                931            
https://reestr.minsvyaz.ru/reestr/75881/

887 of 1000


Рег. номер ПО:
                932            
https://reestr.minsvyaz.ru/reestr/75882/

888 of 1000


Рег. номер ПО:
                933            
https://reestr.minsvyaz.ru/reestr/75883/

889 of 1



Рег. номер ПО:
                988            
https://reestr.minsvyaz.ru/reestr/77508/

944 of 1000


Рег. номер ПО:
                989            
https://reestr.minsvyaz.ru/reestr/77509/

945 of 1000


Рег. номер ПО:
                990            
https://reestr.minsvyaz.ru/reestr/77510/

946 of 1000


Рег. номер ПО:
                991            
https://reestr.minsvyaz.ru/reestr/77511/

947 of 1000
АИС УОС ФУО

Рег. номер ПО:
                992            
https://reestr.minsvyaz.ru/reestr/77512/

948 of 1000
Составление и исполнение доходов и расходов бюджетов субъектов, ЗАТО и муниципальных образований (Бюджет-Web)

Рег. номер ПО:
                993            
https://reestr.minsvyaz.ru/reestr/77513/

949 of 1000
Подсистема учета целевых программ и формирования иерархического дерева целеполагания (Целевые программы-СМАРТ)::Целевые программы - КС

Рег. номер ПО:
                994            
https://reestr.minsvyaz.ru/reestr/77514/

950 of 1000
Составление и исполнение 

In [12]:
values

{'Название по': ['Система управления базами данных  «Ред База Данных»',
  '1С:Школа. Информатика, 11 класс',
  '1С:Школа. Русский язык, 5–6 класс. Лексикология'],
 'Вид организации': ['российская коммерческая организация',
  'российская коммерческая организация',
  'российская коммерческая организация'],
 'Название организации': ['ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "РЕД СОФТ"',
  'Общество с ограниченной ответственностью "1С-Паблишинг"',
  'Общество с ограниченной ответственностью "1С-Паблишинг"'],
 'Все продукты организации': ['https://reestr.digital.gov.ru/reestr/?filter_owner=56139',
  'https://reestr.digital.gov.ru/reestr/?filter_owner=55800',
  'https://reestr.digital.gov.ru/reestr/?filter_owner=55800'],
 'ИНН': ['9705000373', '7725192493', '7725192493'],
 'Сайт': ['http://reddatabase.ru/documentation/',
  'http://obr.1c.ru/educational/uchenikam/informatika-11-kl-2-e-izd-/',
  'http://obr.1c.ru/educational/uchenikam/russkiy-yazyk-5-6-kl-leksikologiya/'],
 'Альтернатив': ['СУ

In [19]:
df = pd.DataFrame(values)
df

,Название по,Ссылка на ПО,Вид организации,Название организации,Все продукты организации,ИНН,Сведения об исключительном праве,Сайт,Альтернатив,Класс по,Дата регистрации,Рег.номер ПО,Дата решения,Решение,Ссылка на приказ
0,Система управления базами данных «Ред База Да...,https://reestr.minsvyaz.ru/reestr/58191/\n,российская коммерческая организация,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЕД ...",https://reestr.digital.gov.ru/reestr/?filter_o...,9705000373,приобретение исключительного права,http://reddatabase.ru/documentation/,СУБД «Ред База Данных»,Системы управления базами данных,29.01.2016,1,29.01.2016,Приказ Минкомсвязи России от 29.01.2016 №19,https://reestr.minsvyaz.ru/upload/iblock/88a/1...
1,"1С:Школа. Информатика, 11 класс",https://reestr.minsvyaz.ru/reestr/58193/\n,российская коммерческая организация,"Общество с ограниченной ответственностью ""1С-П...",https://reestr.digital.gov.ru/reestr/?filter_o...,7725192493,Собственная разработка,http://obr.1c.ru/educational/uchenikam/informa...,,Системы управления процессами организации::Инф...,29.01.2016,2,29.01.2016,Приказ Минкомсвязи России от 29.01.2016 №19,https://reestr.minsvyaz.ru/upload/iblock/88a/1...
2,"1С:Школа. Русский язык, 5–6 класс. Лексикология",https://reestr.minsvyaz.ru/reestr/58192/\n,российская коммерческая организация,"Общество с ограниченной ответственностью ""1С-П...",https://reestr.digital.gov.ru/reestr/?filter_o...,7725192493,Собственная разработка,http://obr.1c.ru/educational/uchenikam/russkiy...,,Системы управления процессами организации::Инф...,29.01.2016,3,29.01.2016,Приказ Минкомсвязи России от 29.01.2016 №19,https://reestr.minsvyaz.ru/upload/iblock/88a/1...
3,1С:Комплексная автоматизация 8,https://reestr.minsvyaz.ru/reestr/61242/\n,российская коммерческая организация,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""1С-С...",https://reestr.digital.gov.ru/reestr/?filter_o...,7730643014,Договор об отчуждении исключительного права на...,http://v8.1c.ru/ka/,,Системы управления процессами организации,20.02.2016,5,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...
4,"Электронный периодический справочник ""Система ...",https://reestr.minsvyaz.ru/reestr/61245/\n,российская коммерческая организация,"ООО ""Научно-производственное предприятие ""ГАРА...",https://reestr.digital.gov.ru/reestr/?filter_o...,7706131706,Свидетельство о государственной регистрации ба...,http://www.aero.garant.ru,"ЭПС ""Система ГАРАНТ""",Информационные системы для решения специфическ...,20.02.2016,6,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Интерактивное учебное пособие ""Наглядная школа...",https://reestr.minsvyaz.ru/reestr/77559/\n,российская коммерческая организация,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭКЗА...",https://reestr.digital.gov.ru/reestr/?filter_o...,7701881818,собственная разработка (создание служебного пр...,http://examen-media.ru/naglyadnaya-shkola-1-4-...,,Информационные системы для решения специфическ...,1.06.2016,1040,31.05.2016,Приказ Минкомсвязи России от 31.05.2016 №226,https://reestr.minsvyaz.ru/upload/iblock/795/2...
996,"Интерактивное учебное пособие ""Наглядная школа...",https://reestr.minsvyaz.ru/reestr/77560/\n,российская коммерческая организация,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭКЗА...",https://reestr.digital.gov.ru/reestr/?filter_o...,7701881818,собственная разработка (создание служебного пр...,http://examen-media.ru/naglyadnaya-shkola-1-4-...,,Информационные системы для решения специфическ...,1.06.2016,1041,31.05.2016,Приказ Минкомсвязи России от 31.05.2016 №226,https://reestr.minsvyaz.ru/upload/iblock/795/2...
997,"Интерактивное учебное пособие ""Наглядная школа...",https://reestr.minsvyaz.ru/reestr/77561/\n,российская коммерческая организация,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭКЗА...",https://reestr.digital.gov.ru/reestr/?filter_o...,7701881818,

In [20]:

#запись датафрейма в csv-файл с кодировкой utf-8
df.to_excel('Reestr.xlsx', index = False)


In [16]:
req = requests.get('https://reestr.digital.gov.ru/reestr/61319/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
altname = str(soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div')).split('<br/>')
altname = [x.strip() for x in altname]
altname = re.sub('''<div style="margin-bottom: 16px;">\n<span style="font-size: 15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>''','',
                         altname[0]).strip() +'::'+ "::".join(altname[1:-1])
altname = re.sub('</div>','', altname.strip('::')).strip()
altname

'Dr.Web SSS::SSS::Dr.Web Server Security Suite (macOS) версия 12::Dr.Web Server Security Suite (для Windows) версия 11::Dr.Web Server Security Suite (для Unix) версия 11::Dr.Web Server Security Suite (Windows) версия 12'

In [15]:
altname = str(soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div')).split('<br/>')
altname = [x.strip() for x in altname]
altname

['<div style="margin-bottom: 16px;">\n<span style="font-size: 15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>\n\t\t\t\t\tDr.Web SSS',
 'SSS',
 'Dr.Web Server Security Suite (macOS) версия 12',
 'Dr.Web Server Security Suite (для Windows) версия 11',
 'Dr.Web Server Security Suite (для Unix) версия 11',
 'Dr.Web Server Security Suite (Windows) версия 12',
 '</div>']

In [10]:
#преобразование словаря в датафрейм
values['Рег.номер ПО']

['2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2']

In [58]:
pd.DataFrame(values)

,Название по,Вид организации,Название организации,Все продукты организации,ИНН,Сайт,Альтернатив,Класс по,Дата регистрации,Рег.номер ПО,Дата решения,Решение,Ссылка на приказ,Сведения об исключительном праве
0,Система управления базами данных «Ред База Да...,российская коммерческая организация,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЕД ...",https://reestr.digital.gov.ru/reestr/?filter_o...,9705000373,http://reddatabase.ru/documentation/,СУБД «Ред База Данных»,Системы управления базами данных,29.01.2016,1,29.01.2016,Приказ Минкомсвязи России от 29.01.2016 №19,https://reestr.minsvyaz.ru/upload/iblock/88a/1...,Собственная разработка.\n \n Программный проду...
1,"1С:Школа. Информатика, 11 класс",российская коммерческая организация,"Общество с ограниченной ответственностью ""1С-П...",https://reestr.digital.gov.ru/reestr/?filter_o...,7725192493,http://obr.1c.ru/educational/uchenikam/informa...,,Системы управления процессами организации::Инф...,29.01.2016,2,29.01.2016,Приказ Минкомсвязи России от 29.01.2016 №19,https://reestr.minsvyaz.ru/upload/iblock/88a/1...,Собственная разработка.\n \n Программный проду...
2,"1С:Школа. Русский язык, 5–6 класс. Лексикология",российская коммерческая организация,"Общество с ограниченной ответственностью ""1С-П...",https://reestr.digital.gov.ru/reestr/?filter_o...,7725192493,http://obr.1c.ru/educational/uchenikam/russkiy...,,Системы управления процессами организации::Инф...,29.01.2016,3,29.01.2016,Приказ Минкомсвязи России от 29.01.2016 №19,https://reestr.minsvyaz.ru/upload/iblock/88a/1...,Собственная разработка.\n \n Программный проду...
3,1С:Комплексная автоматизация 8,российская коммерческая организация,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""1С-С...",https://reestr.digital.gov.ru/reestr/?filter_o...,7730643014,http://v8.1c.ru/ka/,,Системы управления процессами организации,20.02.2016,5,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...,Собственная разработка.\n \n Программный проду...
4,"Электронный периодический справочник ""Система ...",российская коммерческая организация,"ООО ""Научно-производственное предприятие ""ГАРА...",https://reestr.digital.gov.ru/reestr/?filter_o...,7706131706,http://www.aero.garant.ru,"ЭПС ""Система ГАРАНТ""",Информационные системы для решения специфическ...,20.02.2016,6,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...,Собственная разработка.\n \n Программный проду...
5,Система электронного документооборота и автома...,российская коммерческая организация,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""КОГНИТИВ""",https://reestr.digital.gov.ru/reestr/?filter_o...,7728742674,http://www.evfrat.ru,Е1 Евфрат,Офисные приложения::Прикладное программное обе...,20.02.2016,8,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...,Собственная разработка.\n \n Программный проду...
6,Альт Линукс СПТ,российская коммерческая организация,"Общество с ограниченной ответственностью ""Своб...",https://reestr.digital.gov.ru/reestr/?filter_o...,5003088997,http://sptlinux.ru/,"Альт Линукс СПТ, сертификат ФСТЭК",Операционные системы,20.02.2016,9,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...,Собственная разработка.\n \n Программный проду...
7,ABBYY ScanDifFinder SDK,"российская коммерческая организация, имеющая в...","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АБИ ...",https://reestr.digital.gov.ru/reestr/?filter_o...,7723594937,http://www.abbyy.ru/scandiffinder/,,Библиотеки подпрограмм (SDK)::Лингвистическое ...,20.02.2016,10,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...,Собственная разработка.\n \n Программный проду...
8,ABBYY PassportReader SDK,"российская коммерческая организация, имеющая в...","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АБИ ...",https://reestr.digital.gov.ru/reestr/?filter_o...,7723594937,http://www.abbyy.ru/p

In [7]:
req = requests.get('https://reestr.digital.gov.ru/reestr/61265/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
altname = soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div')
list_ = str(altname).split('<br/>')
string = re.sub('Альтернативные наименования:', '', altname.text) + ",".join(list_[0:-1])
string.strip()

'1С-Битрикс24: Корпоративный портал - Холдинг1С-Битрикс24 (Холдинг)1С-Битрикс24«1С-Битрикс24». Лицензия Энтерпрайз <div style="margin-bottom: 16px;">\n<span style="font-size: 15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>\n\t\t\t\t\t1С-Битрикс24: Корпоративный портал - Холдинг,1С-Битрикс24 (Холдинг),1С-Битрикс24,«1С-Битрикс24». Лицензия Энтерпрайз'

In [17]:
req = requests.get('https://reestr.digital.gov.ru/reestr/61265/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
altname = soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div')
list_ = str(altname).split('<br/>')
re.sub('<div style="margin-bottom: 16px;">\n<span style="font-size: '
       '15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>\n\t\t\t\t\t',
       '',",".join(list_))

'1С-Битрикс24: Корпоративный портал - Холдинг,1С-Битрикс24 (Холдинг),1С-Битрикс24,«1С-Битрикс24». Лицензия Энтерпрайз, </div>'

In [15]:
req = requests.get('https://reestr.digital.gov.ru/reestr/58191/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
altname = soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div')
list_ = str(altname).split('<br/>')
list_

['<div style="margin-bottom: 16px;">\n<span style="font-size: 15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>\n\t\t\t\t\tСУБД «Ред База Данных»\t\t\t\t</div>']

In [67]:
re.sub('Альтернативные наименования:', '', altname.text)

'\n\n\t\t\t\t\t1С-Битрикс24: Корпоративный портал - Холдинг1С-Битрикс24 (Холдинг)1С-Битрикс24«1С-Битрикс24». Лицензия Энтерпрайз '

In [29]:
#ЭТО РАБОТАЕТ!!!!!
req = requests.get('https://reestr.digital.gov.ru/reestr/58193/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
altname = soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div').children
li = [str(x).strip() for x in altname]
li = list(filter(lambda x: x!= '<br/>', li))
re.sub('<span style="font-size: 15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>',
       '',','.join(li)).strip(',')

AttributeError: 'NoneType' object has no attribute 'find_parent'

In [41]:

req = requests.get('https://reestr.digital.gov.ru/reestr/58193/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
softclas = soup.find('span', text=re.compile("Класс ПО:")).find_parent('div').find_all('font')
softclass = '::'.join([x.text for x in softclas])

'Системы управления процессами организации::Информационные системы для решения специфических отраслевых задач'

In [44]:
svedeniya = soup.find('p', attrs = {'style':"line-height: 28px;"}).text.strip()

'Собственная разработка'

In [12]:
import pandas as pd
values_ = {'a':[1,2,3,4,5],'b':[1,2,3,4,5],'c':[4,5,6,7,8]}
df_ = pd.DataFrame(values_)
df_.to_excel('values__.xlsx')